# Button

> [Buttons](https://daisyui.com/components/button/) allow the user to take actions or make choices.

In [ ]:
#| default_exp actions.button

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Union, List, Any, Literal
from enum import Enum
from dataclasses import dataclass, field
from fasthtml.common import *
from cjm_tailwind_utils.all import TailwindBuilder
from cjm_fasthtml_daisyui.core.base import DaisyComponent, DaisySize
from cjm_fasthtml_daisyui.core.colors import (
    SemanticColor, ColorUtility, ColorBuilder, apply_semantic_colors
)
from cjm_fasthtml_daisyui.core.behaviors import InteractiveMixin, FormControlMixin
from cjm_fasthtml_daisyui.core.variants import HasVariants, StyleType, create_style_variant
from cjm_fasthtml_daisyui.core.htmx import HTMXComponent, HTMXAttrs

## Button Component

The Button component is one of the most versatile components in daisyUI, supporting:
- Multiple color variants (primary, secondary, accent, etc.)
- Multiple styles (solid, outline, ghost, link, etc.)
- Size variations (xs to xl)
- Shape modifiers (square, circle, wide, block)
- States (active, disabled, loading)
- Icons and content composition

In [ ]:
#| export
class ButtonShape(str, Enum):
    """Button shape modifiers"""
    DEFAULT = ""
    WIDE = "wide"      # Extra horizontal padding
    BLOCK = "block"    # Full width button
    SQUARE = "square"  # Square button (equal width/height)
    CIRCLE = "circle"  # Circular button

In [ ]:
#| export
@dataclass
class Btn(HTMXComponent, HasVariants, InteractiveMixin, FormControlMixin):
    """
    daisyUI Button component with full feature support.
    
    Supports all button variants, styles, sizes, shapes, and states.
    Can be used as a regular button, submit button, or link button.
    
    Examples:
        Basic button:
            Btn("Click me", color=SemanticColor.PRIMARY)
        
        Icon button:
            Btn(Icon("heart"), shape=ButtonShape.CIRCLE, style=StyleType.GHOST)
        
        Loading button:
            Btn("Submit", loading=True, disabled=True)
    """
    
    # Content
    children: List[Any] = field(default_factory=list)
    
    # Button specific
    shape: Optional[ButtonShape] = None
    no_animation: bool = False  # Disable click animation
    
    # Icon support
    icon_start: Optional[FT] = None
    icon_end: Optional[FT] = None
    
    # Link button support
    href: Optional[str] = None
    target: Optional[str] = None
    
    # Form button properties
    type: Optional[str] = None
    form: Optional[str] = None
    
    # Style property for convenient access
    style: Optional[Union[StyleType, str]] = None
    
    def __init__(self, *children, **kwargs):
        "TODO: Add function description"
        # Extract children from args
        self.children = list(children)
        
        # Extract button-specific attributes before parent init
        self.icon_start = kwargs.pop('icon_start', None)
        self.icon_end = kwargs.pop('icon_end', None)
        self.shape = kwargs.pop('shape', None)
        self.no_animation = kwargs.pop('no_animation', False)
        self.href = kwargs.pop('href', None)
        self.target = kwargs.pop('target', None)
        
        # Extract style and handle variant
        self.style = kwargs.pop('style', None)
        
        # Initialize variant values for HasVariants
        self.variant_values = kwargs.pop('variant_values', {})
        
        # Extract InteractiveMixin properties
        self.active = kwargs.pop('active', False)
        self.disabled = kwargs.pop('disabled', False)
        self.loading = kwargs.pop('loading', False)
        self.open = kwargs.pop('open', False)
        self.checked = kwargs.pop('checked', False)
        self.focus = kwargs.pop('focus', False)
        self.hover = kwargs.pop('hover', False)
        
        # Extract FormControlMixin properties
        self.name = kwargs.pop('name', None)
        self.value = kwargs.pop('value', None)
        self.placeholder = kwargs.pop('placeholder', None)
        self.required = kwargs.pop('required', False)
        self.readonly = kwargs.pop('readonly', False)
        self.autofocus = kwargs.pop('autofocus', False)
        self.type = kwargs.pop('type', None)
        self.form = kwargs.pop('form', None)
        
        # Initialize parent classes
        super().__init__(**kwargs)
        
        # Set the style variant if provided
        if self.style:
            style_value = self.style.value if isinstance(self.style, StyleType) else self.style
            self.set_variant("style", style_value)
    
    @classmethod
    def variants(cls) -> Dict[str, Any]:
        """Define available variants for buttons.
        
        Returns:
            Dictionary of variant definitions
        """
        return {
            "style": create_style_variant("btn")
        }
    
    def component_class(
        self
    ) -> str:  # TODO: Add return description
        "TODO: Add function description"
        return "btn"
    
    def supports_color(
        self
    ) -> bool:  # TODO: Add return description
        "TODO: Add function description"
        return True
    
    def supports_size(
        self
    ) -> bool:  # TODO: Add return description
        "TODO: Add function description"
        return True
    
    def supports_glass(
        self
    ) -> bool:  # TODO: Add return description
        "TODO: Add function description"
        return True
    
    def supports_active(
        self
    ) -> bool:  # TODO: Add return description
        "TODO: Add function description"
        return True
    
    def supports_disabled(
        self
    ) -> bool:  # TODO: Add return description
        "TODO: Add function description"
        return True
    
    def supports_loading(
        self
    ) -> bool:  # TODO: Add return description
        "TODO: Add function description"
        return True
    
    def modifier_classes(
        self
    ) -> List[str]:  # TODO: Add return description
        """Build all modifier classes"""
        classes = super().modifier_classes()
        
        # Add shape modifier
        if self.shape and self.shape != ButtonShape.DEFAULT:
            classes.append(f"btn-{self.shape.value}")
        
        # Add behavior classes from InteractiveMixin
        # Note: These are now handled by get_css_classes() from InteractiveMixin
        
        # Add no-animation
        if self.no_animation:
            classes.append("no-animation")
        
        return classes
    
    def render_content(
        self
    ) -> List[FT]:  # TODO: Add return description
        """Render button content with icons"""
        content = []
        
        # Add start icon
        if self.icon_start:
            content.append(self.icon_start)
        
        # Add main content
        content.extend(self.children)
        
        # Add loading spinner if needed
        if self.loading:
            content.append(Span(cls="loading loading-spinner"))
        
        # Add end icon
        if self.icon_end:
            content.append(self.icon_end)
        
        return content
    
    def render_attrs(
        self
    ) -> Dict[str, Any]:  # TODO: Add return description
        """Build all HTML attributes including form and behavior attrs."""
        attrs = super().render_attrs()
        
        # Add behavior attributes
        attrs.update(self.behavior_attrs())
        
        # Add form attributes
        if hasattr(self, 'form_attrs'):
            attrs.update(self.form_attrs())
        
        return attrs
    
    def render(
        self
    ) -> FT:  # TODO: Add return description
        """Render the button element"""
        attrs = self.render_attrs()
        content = self.render_content()
        
        # Determine element type
        if self.href:
            # Link button
            attrs['href'] = self.href
            if self.target:
                attrs['target'] = self.target
            if self.disabled:
                attrs['tabindex'] = '-1'
                attrs['role'] = 'button'
                attrs['aria-disabled'] = 'true'
            return A(*content, **attrs)
        else:
            # Regular or form button
            if self.type:
                attrs['type'] = self.type
            else:
                attrs['type'] = 'button'  # Default to prevent form submission
                
            if self.name:
                attrs['name'] = self.name
            if self.value:
                attrs['value'] = self.value
            if self.form:
                attrs['form'] = self.form
                
            # Use FastHTML's Button element
            return Button(*content, **attrs)
    
    # Convenience methods for common patterns
    @classmethod
    def primary(
        cls,  # TODO: Add type hint and description
        *children,
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create a primary button"""
        return cls(*children, color=SemanticColor.PRIMARY, **kwargs)
    
    @classmethod
    def secondary(
        cls,  # TODO: Add type hint and description
        *children,
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create a secondary button"""
        return cls(*children, color=SemanticColor.SECONDARY, **kwargs)
    
    @classmethod
    def accent(
        cls,  # TODO: Add type hint and description
        *children,
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create an accent button"""
        return cls(*children, color=SemanticColor.ACCENT, **kwargs)
    
    @classmethod
    def success(
        cls,  # TODO: Add type hint and description
        *children,
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create a success button"""
        return cls(*children, color=SemanticColor.SUCCESS, **kwargs)
    
    @classmethod
    def error(
        cls,  # TODO: Add type hint and description
        *children,
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create an error/danger button"""
        return cls(*children, color=SemanticColor.ERROR, **kwargs)
    
    @classmethod
    def warning(
        cls,  # TODO: Add type hint and description
        *children,
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create a warning button"""
        return cls(*children, color=SemanticColor.WARNING, **kwargs)
    
    @classmethod
    def info(
        cls,  # TODO: Add type hint and description
        *children,
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create an info button"""
        return cls(*children, color=SemanticColor.INFO, **kwargs)
    
    @classmethod
    def ghost(
        cls,  # TODO: Add type hint and description
        *children,
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create a ghost button"""
        kwargs['style'] = StyleType.GHOST
        return cls(*children, **kwargs)
    
    @classmethod
    def link(
        cls,  # TODO: Add type hint and description
        *children,
        href: str,
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create a link-styled button"""
        kwargs['style'] = StyleType.LINK
        kwargs['href'] = href
        return cls(*children, **kwargs)
    
    @classmethod
    def outline(
        cls,  # TODO: Add type hint and description
        *children,
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create an outline button"""
        kwargs['style'] = StyleType.OUTLINE
        return cls(*children, **kwargs)
    
    @classmethod
    def icon(
        cls,  # TODO: Add type hint and description
        icon: FT,  # TODO: Add description
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create an icon-only button (typically square or circle)"""
        kwargs.setdefault('shape', ButtonShape.SQUARE)
        return cls(icon, **kwargs)
    
    @classmethod
    def submit(
        cls,  # TODO: Add type hint and description
        text: str = "Submit",  # TODO: Add description
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create a submit button"""
        return cls(text, type="submit", color=SemanticColor.PRIMARY, **kwargs)
    
    @classmethod
    def cancel(
        cls,  # TODO: Add type hint and description
        text: str = "Cancel",  # TODO: Add description
        **kwargs
    ) -> 'Btn':  # TODO: Add return description
        """Create a cancel button"""
        kwargs['style'] = StyleType.GHOST
        return cls(text, **kwargs)

## Usage Examples

Let's test the Button component with various configurations.

### Basic Setup

In [ ]:
#|eval: false
from cjm_fasthtml_daisyui.core.testing import ComponentTester, theme_test
from cjm_fasthtml_daisyui.core.config import DaisyUITheme
from fasthtml.common import Div, H3
from fasthtml.jupyter import HTMX

### Color Variants

In [ ]:
#|eval: false
# Test color variants
tester = ComponentTester("Button Color Variants")

# Using convenience methods
tester.add(
    Div(
        Btn.primary("Primary").render(),
        Btn.secondary("Secondary").render(),
        Btn.accent("Accent").render(),
        Btn.success("Success").render(),
        Btn.warning("Warning").render(),
        Btn.error("Error").render(),
        Btn.info("Info").render(),
        Btn("Neutral", color=SemanticColor.NEUTRAL).render(),
        cls=TailwindBuilder().flex(wrap="wrap").gap(2).build()
    ),
    title="Color Variants",
    description="All semantic color options for buttons"
)

tester.start()
HTMX()

In [ ]:
#|eval: false
tester.stop()

### Style Modifiers

In [ ]:
#|eval: false
# Test style modifiers
tester2 = ComponentTester("Button Style Modifiers")

# Different styles with primary color
tester2.add(
    Div(
        Btn("Default", color=SemanticColor.PRIMARY).render(),
        Btn.outline("Outline", color=SemanticColor.PRIMARY).render(),
        Btn("Dash", color=SemanticColor.PRIMARY, style=StyleType.DASH).render(),
        Btn("Soft", color=SemanticColor.PRIMARY, style=StyleType.SOFT).render(),
        Btn.ghost("Ghost", color=SemanticColor.PRIMARY).render(),
        Btn.link("Link", href="#", color=SemanticColor.PRIMARY).render(),
        cls=TailwindBuilder().flex(wrap="wrap").gap(2).build()
    ),
    title="Style Modifiers",
    description="Different button styles available",
    code='''Button("Default", color=SemanticColor.PRIMARY)
Button.outline("Outline", color=SemanticColor.PRIMARY)
Button("Dash", color=SemanticColor.PRIMARY, style=StyleType.DASH)
Button("Soft", color=SemanticColor.PRIMARY, style=StyleType.SOFT)
Button.ghost("Ghost", color=SemanticColor.PRIMARY)
Button.link("Link", href="#", color=SemanticColor.PRIMARY)'''
)

tester2.start()  # Uncomment to run
HTMX()

In [ ]:
#|eval: false
tester2.stop()

### Sizes and Shapes

In [ ]:
#|eval: false
# Test sizes and shapes
tester3 = ComponentTester("Button Sizes and Shapes")

# Size variations
tester3.add(
    Div(
        Btn("XS", size=DaisySize.XS).render(),
        Btn("SM", size=DaisySize.SM).render(),
        Btn("MD", size=DaisySize.MD).render(),
        Btn("LG", size=DaisySize.LG).render(),
        Btn("XL", size=DaisySize.XL).render(),
        cls=TailwindBuilder().flex().items("center").gap(2).build()
    ),
    title="Size Variations"
)

# Shape variations
tester3.add(
    Div(
        Btn("Default").render(),
        Btn("Wide", shape=ButtonShape.WIDE).render(),
        Btn.icon("□", shape=ButtonShape.SQUARE).render(),
        Btn.icon("●", shape=ButtonShape.CIRCLE).render(),
        cls=TailwindBuilder().flex().items("center").gap(2).build()
    ),
    title="Shape Variations"
)

# Block button
tester3.add(
    Btn("Block Button (Full Width)", shape=ButtonShape.BLOCK).render(),
    title="Block Button"
)

tester3.start()  # Uncomment to run
HTMX()

In [ ]:
#|eval: false
tester3.stop()

### States and Behaviors

In [ ]:
#|eval: false
# Test states
tester4 = ComponentTester("Button States")

# Interactive states
tester4.add(
    Div(
        Btn("Normal").render(),
        Btn("Active", active=True).render(),
        Btn("Disabled", disabled=True).render(),
        Btn("Loading", loading=True).render(),
        Btn("No Animation", no_animation=True).render(),
        cls=TailwindBuilder().flex(wrap="wrap").gap(2).build()
    ),
    title="Button States"
)

# Glass effect
tester4.add(
    Div(
        Btn("Glass Effect", glass=True, color=SemanticColor.PRIMARY).render(),
        Btn("Glass Ghost", glass=True, style=StyleType.GHOST).render(),
        cls=TailwindBuilder().flex().gap(2).build()
    ),
    title="Glass Effect",
    description="Buttons with glass morphism effect"
)

tester4.start()  # Uncomment to run
HTMX()

In [ ]:
#|eval: false
tester4.stop()

### Buttons with Icons

In [ ]:
#|eval: false
# Test buttons with icons
# Note: Using simple text icons for demo - in real apps use SVG icons
tester5 = ComponentTester("Buttons with Icons")

# Icon positions
tester5.add(
    Div(
        Btn("Download", icon_start="⬇").render(),
        Btn("Upload", icon_end="⬆").render(),
        Btn("Settings", icon_start="⚙", icon_end="▶").render(),
        cls=TailwindBuilder().flex().gap(2).build()
    ),
    title="Icon Positions"
)

# Icon-only buttons
tester5.add(
    Div(
        Btn.icon("❤", color=SemanticColor.ERROR).render(),
        Btn.icon("⭐", shape=ButtonShape.CIRCLE, color=SemanticColor.WARNING).render(),
        Btn.icon("🔧", shape=ButtonShape.SQUARE, style=StyleType.GHOST).render(),
        Btn.icon("✖", shape=ButtonShape.CIRCLE, size=DaisySize.SM, style=StyleType.OUTLINE).render(),
        cls=TailwindBuilder().flex().gap(2).build()
    ),
    title="Icon-Only Buttons"
)

tester5.start()  # Uncomment to run
HTMX()

In [ ]:
#|eval: false
tester5.stop()

### HTMX Integration

In [ ]:
#|eval: false
# Test HTMX features
tester7 = ComponentTester("HTMX Button Examples")

# HTMX-enabled button
htmx_btn = Btn("Load Data", color=SemanticColor.PRIMARY)
htmx_btn.with_htmx(
    get="/api/data",
    target="#results",
    swap="innerHTML"
).with_loading("loading-indicator")

# Better approach using the color system
tb_results = TailwindBuilder()
tb_results.m(4, "t").p(4).rounded()
# Add semantic colors properly
tb_results.add_class(apply_semantic_colors(bg=SemanticColor.BASE_200))

tester7.add(
    Div(
        htmx_btn.render(),
        Div(id="results", cls=tb_results.build()),
        Div(id="loading-indicator", cls="htmx-indicator", children=["Loading..."])
    ),
    title="HTMX Button",
    description="Button that loads data via HTMX",
    code='''htmx_btn = Button("Load Data", color=SemanticColor.PRIMARY)
htmx_btn.with_htmx(
    get="/api/data",
    target="#results",
    swap="innerHTML"
).with_loading("loading-indicator")'''
)

# Form submission button
submit_btn = Btn.submit("Submit Form")
submit_btn.with_htmx(
    post="/api/submit",
    trigger="click",
    include="form"
).with_confirmation("Are you sure you want to submit?")

tester7.add(
    submit_btn.render(),
    title="HTMX Form Submit",
    description="Submit button with confirmation"
)

tester7.start()  # Uncomment to run
HTMX()

In [ ]:
#|eval: false
tester7.stop()

### Responsive Buttons

In [ ]:
#|eval: false
# Test responsive features
tester8 = ComponentTester("Responsive Buttons")

# Responsive size
responsive_btn = Btn(
    "Responsive Size",
    size=DaisySize.SM,  # Default small
    responsive_size={"md": "md", "lg": "lg"},  # Medium on md+, large on lg+
    color=SemanticColor.PRIMARY
)

# Responsive visibility
mobile_btn = Btn(
    "Mobile Only",
    responsive_hide=["md"],  # Hide on medium screens and up
    color=SemanticColor.SECONDARY
)

desktop_btn = Btn(
    "Desktop Only", 
    responsive_show=["md"],  # Show on medium screens and up
    cls="hidden",  # Hidden by default
    color=SemanticColor.ACCENT
)

tester8.add(
    Div(
        responsive_btn.render(),
        mobile_btn.render(),
        desktop_btn.render(),
        cls=TailwindBuilder().flex().gap(2).build()
    ),
    title="Responsive Behaviors",
    description="Resize browser to see responsive changes"
)

tester8.start()  # Uncomment to run
HTMX()

In [ ]:
#|eval: false
tester8.stop()

### Complete Example

In [ ]:
#|eval: false
# Complete button showcase
with theme_test([DaisyUITheme.LIGHT, DaisyUITheme.DARK, DaisyUITheme.CUPCAKE]) as tester:
    # Color variants section
    tester.add(
        Div(
            H3("Color Variants", cls=TailwindBuilder().text(size="lg", weight="bold").m(2, "b").build()),
            Div(
                *[Btn(color.name.title(), color=color).render() for color in SemanticColor if "_" not in color.name],
                cls=TailwindBuilder().flex(wrap="wrap").gap(2).build()
            ),
            cls=TailwindBuilder().space(2, "y").build()
        ),
        title="Complete Button Showcase"
    )
    
    # Styles section  
    tester.add(
        Div(
            H3("Button Styles", cls=TailwindBuilder().text(size="lg", weight="bold").m(2, "b").build()),
            Div(
                Btn("Default", color=SemanticColor.PRIMARY).render(),
                Btn("Outline", color=SemanticColor.PRIMARY, style=StyleType.OUTLINE).render(),
                Btn("Dash", color=SemanticColor.PRIMARY, style=StyleType.DASH).render(),
                Btn("Soft", color=SemanticColor.PRIMARY, style=StyleType.SOFT).render(),
                Btn("Ghost", color=SemanticColor.PRIMARY, style=StyleType.GHOST).render(),
                Btn("Link", color=SemanticColor.PRIMARY, style=StyleType.LINK).render(),
                cls=TailwindBuilder().flex(wrap="wrap").gap(2).build()
            ),
            cls=TailwindBuilder().space(2, "y").m(4, "t").build()
        ),
        title="Style Variations"
    )
    
    # Real-world examples
    tester.add(
        Div(
            H3("Real-World Examples", cls=TailwindBuilder().text(size="lg", weight="bold").m(2, "b").build()),
            # Form actions using flex layout
            Div(
                Btn.cancel().render(),
                Btn.submit("Save Changes").render(),
                cls=TailwindBuilder().flex().gap(2).justify("end").build()
            ),
            # Social buttons
            Div(
                Btn("Login with Google", icon_start="🔍", shape=ButtonShape.WIDE).render(),
                Btn("Share", icon_end="↗", style=StyleType.OUTLINE).render(),
                cls=TailwindBuilder().flex().gap(2).m(4, "t").build()
            ),
            # Loading states
            Div(
                Btn("Processing...", loading=True, disabled=True).render(),
                Btn("Download", icon_start="⬇", color=SemanticColor.SUCCESS).render(),
                cls=TailwindBuilder().flex().gap(2).m(4, "t").build()
            ),
            cls=TailwindBuilder().space(4, "y").m(4, "t").build()
        ),
        title="Common Patterns"
    )

# Pass the tester to start() to run the server
# The 'with' statement will automatically call start()
HTMX()

In [ ]:
#|eval: false
tester.stop()

## Best Practices: Using Colors with TailwindBuilder

When combining TailwindBuilder with daisyUI semantic colors, use the color system from `cjm_fasthtml_daisyui.core.colors`:

In [ ]:
#|eval: false
# Best practices for combining TailwindBuilder with daisyUI colors

# 1. For standard Tailwind colors, use TailwindBuilder methods
tb1 = TailwindBuilder()
tb1.bg("blue-500").text("white").p(4).rounded()
standard_result = tb1.build()
print(f"Standard Tailwind: {standard_result}")

# 2. For daisyUI semantic colors, use the color system
from cjm_fasthtml_daisyui.core.colors import apply_semantic_colors, ColorUtility, SemanticColor

tb2 = TailwindBuilder()
tb2.p(4).rounded()
# Add semantic colors using apply_semantic_colors
tb2.add_class(apply_semantic_colors(bg=SemanticColor.PRIMARY))
semantic_result = tb2.build()
print(f"Semantic colors: {semantic_result}")

# 3. For complex layouts with mixed colors
tb3 = TailwindBuilder()
tb3.flex().gap(2).p(4).rounded("lg")
# Add daisyUI surface colors
tb3.add_class(apply_semantic_colors(
    bg=SemanticColor.BASE_200,
    border=SemanticColor.NEUTRAL
))
complex_result = tb3.build()
print(f"Complex with mixed: {complex_result}")

# 4. Using ColorBuilder for semantic color combinations
from cjm_fasthtml_daisyui.core.colors import ColorBuilder

color_builder = ColorBuilder()
button_colors = color_builder.brand_primary().border(SemanticColor.PRIMARY).build()

tb4 = TailwindBuilder()
tb4.p(2, "x").p(1, "y").rounded().add_class(button_colors)
button_result = tb4.build()
print(f"Button with ColorBuilder: {button_result}")

Standard Tailwind: bg-blue-500 text-white p-4 rounded
Semantic colors: p-4 rounded bg-primary text-primary-content
Complex with mixed: flex gap-2 p-4 rounded-lg bg-base-200 text-base-content border-neutral
Button with ColorBuilder: px-2 py-1 rounded bg-primary text-primary-content border-primary


## FastHTML Route Examples

Examples of using buttons in FastHTML applications.

```python
from fasthtml.common import *
from cjm_tailwind_utils.all import TailwindBuilder
from cjm_fasthtml_daisyui.actions.button import Btn

app, rt = fast_app()

@rt('/')
def index():
    return Titled("Button Examples",
        # Basic form with buttons
        Form(
            Input(name="email", type="email", placeholder="Email", cls="input input-bordered"),
            Div(
                Btn.cancel(),
                Btn.submit("Sign Up"),
                cls=TailwindBuilder().flex().gap(2).justify("end").build()
            ),
            method="post",
            action="/signup",
            cls=TailwindBuilder().space(4, "y").max_w("md").build()
        )
    )

@rt('/signup')
async def signup(email: str):
    # Process signup
    return Card(
        P(f"Welcome {email}!"),
        Btn.primary("Continue", href="/dashboard")
    )

@rt('/api/data')
def load_data():
    # HTMX endpoint
    return Ul(
        *[Li(f"Item {i}") for i in range(5)],
        cls=TailwindBuilder().add_class("list-disc").p(5, "l").build()
    )

serve()
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()